In [15]:
import pandas as pd
import numpy as np
import re
from scipy.stats import zscore
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [12]:
file_path = r"C:\Users\SAAD COMMUNICATION\OneDrive\Desktop\forecasting_case_study.csv"
df = pd.read_csv(file_path)

In [13]:
df.head()

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,02/05/2017,27750,0%,0,0,0,0.0,0,0,0,0
1,SKU1,02/12/2017,29023,0%,1,0,1,0.0,0,1,0,0
2,SKU1,2/19/2017,45630,17%,0,0,0,0.0,0,0,0,0
3,SKU1,2/26/2017,26789,0%,1,0,1,0.0,0,0,0,0
4,SKU1,03/05/2017,41999,17%,0,0,0,0.0,0,0,0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product             1218 non-null   object 
 1   date                1218 non-null   object 
 2   Sales               1218 non-null   int64  
 3   Price Discount (%)  1218 non-null   object 
 4   In-Store Promo      1218 non-null   int64  
 5   Catalogue Promo     1218 non-null   int64  
 6   Store End Promo     1218 non-null   int64  
 7   Google_Mobility     1218 non-null   float64
 8   Covid_Flag          1218 non-null   int64  
 9   V_DAY               1218 non-null   int64  
 10  EASTER              1218 non-null   int64  
 11  CHRISTMAS           1218 non-null   int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 114.3+ KB


In [16]:
# Convert Date Column to Standard Format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

Handling Missing Values

In [17]:
# Mean Imputation
df['Sales'].fillna(df['Sales'].mean(), inplace=True)

In [22]:
df[['Sales']].head()

,Sales
0,27750
1,29023
2,45630
3,26789
4,41999


In [19]:
# Remove '%' and convert column to numeric
df['Price Discount (%)'] = df['Price Discount (%)'].str.rstrip('%').astype(float)

In [20]:
# KNN Imputation
imputer = KNNImputer(n_neighbors=5)
df[['Price Discount (%)']] = imputer.fit_transform(df[['Price Discount (%)']])

In [23]:
df[['Price Discount (%)']].head()

,Price Discount (%)
0,0.0
1,0.0
2,17.0
3,0.0
4,17.0


Handling Outliers

In [24]:
# Z-Score Method
z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))
df = df[(z_scores < 3).all(axis=1)]

In [25]:
# Inter-Quartile Method
Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['Sales'] >= lower_bound) & (df['Sales'] <= upper_bound)]

NLP Data Cleaning & Featurization

In [26]:
# Remove special characters, convert to lowercase
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df['Product'] = df['Product'].apply(clean_text)

In [27]:
# Categorical Text to Numerical Representations
encoder = LabelEncoder()
df['Product'] = encoder.fit_transform(df['Product'])

Save Cleaned Data

In [28]:
cleaned_file = r"C:\Users\SAAD COMMUNICATION\OneDrive\Desktop\forecasting_case_study.csv"
df.to_csv(cleaned_file, index=False)
print(f"Cleaned data saved at: {cleaned_file}")

Cleaned data saved at: C:\Users\SAAD COMMUNICATION\OneDrive\Desktop\forecasting_case_study.csv
